In [ ]:
!pip install boto3
import os
import time
import requests
import boto3
from datetime import datetime
from typing import Optional

In [ ]:
APIFY_TOKEN   = os.getenv("APIFY_TOKEN", "i")
RUN_ID        = os.getenv("RUN_ID", "w")
EXPORT_FORMAT = os.getenv("EXPORT_FORMAT", "json")
OUT_FILE      = os.getenv("OUT_FILE", "")

AWS_BUCKET = os.getenv("AWS_BUCKET", "capstone-raw-twitterdata-cs668")
AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
S3_KEY     = os.getenv("S3_KEY", "")

WAIT_IF_NOT_DONE = True
POLL_SECS = 6
TIMEOUT_MINS = 45

AWS_ACCESS_KEY_ID     = "AKIAR5WUAXE"
AWS_SECRET_ACCESS_KEY = "anA423Ym1xgrsonr"
AWS_SESSION_TOKEN     = ""

os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
if AWS_SESSION_TOKEN:
    os.environ["AWS_SESSION_TOKEN"] = AWS_SESSION_TOKEN
os.environ["AWS_DEFAULT_REGION"] = AWS_REGION



In [ ]:
def get_run(run_id: str, token: str) -> dict:
    r = requests.get(f"https://api.apify.com/v2/actor-runs/{run_id}",  #fetching data from Apify runs
                     params={"token": token}, timeout=30)
    r.raise_for_status()
    return r.json()["data"]

def export_dataset_stream(dataset_id: str, token: str, export_format: str, clean=True, bom=False, fields: Optional[list]=None):
    base = f"https://api.apify.com/v2/datasets/{dataset_id}/items".
    params = {"token": token, "format": export_format,
              "clean": str(clean).lower(), "bom": str(bom).lower()}
    if fields:
        params["fields"] = ",".join(fields)
    resp = requests.get(base, params=params, stream=True, timeout=300)
    resp.raise_for_status()
    return resp

def save_stream_to_file(resp, filepath: str):
    with open(filepath, "wb") as f:
        for chunk in resp.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)

def upload_stream_to_s3(resp, bucket: str, key: str, region: str = "us-east-1"):
    s3 = boto3.client("s3", region_name=region)
    s3.upload_fileobj(resp.raw, bucket, key)

In [ ]:
run = get_run(RUN_ID, APIFY_TOKEN)

if run["status"] != "SUCCEEDED":
    raise RuntimeError(f"Run {RUN_ID} not SUCCEEDED (status={run['status']}).")

dataset_id = run["defaultDatasetId"]
finished_at = (run.get("finishedAt") or "")[:10] or datetime.utcnow().strftime("%Y-%m-%d")
print("Dataset ID:", dataset_id)

out_file = OUT_FILE or f"apify_run_{RUN_ID}.{EXPORT_FORMAT}"
s3_key   = S3_KEY   or f"raw/run={RUN_ID}-dt={finished_at}.{EXPORT_FORMAT}"  #file name

with export_dataset_stream(dataset_id, APIFY_TOKEN, EXPORT_FORMAT) as resp:
    if AWS_BUCKET:
        print(f"Uploading to s3://{AWS_BUCKET}/{s3_key} ...")
        upload_stream_to_s3(resp, AWS_BUCKET, s3_key, region=AWS_REGION)
        print(f"✅ Uploaded to s3://{AWS_BUCKET}/{s3_key}")

Dataset ID: hl2yN4XSdn
Uploading to s3://capstone-raw-twitterdata-cs668/raw/run=w402mc7R8885YMfEy-dt=2025-09-21.json ...
✅ Uploaded to s3://capstone-raw-twitterdata-cs668/raw/run=w402mc7R8885YMfEy-dt=2025-09-21.json
